# LSTM Sentiment Analysis with Keras
How can we take a bunch of movie reviews from IMDB and use code to classify whether or not the review is positive or negative?

This notebook goes along with the code in [Learning Intelligence 25].

For further reference, check out the [Keras example](https://github.com/keras-team/keras/blob/master/examples/imdb_lstm.py).

## What is Sentiment Analysis?

Sentiment analysis looks at a body of information and decides whether it's good or bad.

For example, IMDB is a movie review site with millions of reviews. However, some reviewers don't leave star ratings. How could IMDB automatically assign star ratings to reviews? Sentiment analysis.

What words could you look for in a review which determine whether it's good or bad?

Perhaps looking for the word 'great' in a review would lead to a high rating.

"This movie was great, if you think lemon juice in the eyes is great."

Not so fast. This is one of the hard problems of natural language processing (NLP), taking text or language in its natural form and analysing it.

Despite this hard problem, using Keras we can quickly build a model which achieves around 90% accuracy on predicting whether a sentiment is good or bad.

## What is Keras?

Keras is a deep learning library used to build deep learning models quickly.

Keras is based on Python.

Because deep learning is a very empircal science (lots of trial and error) Keras is great for building an initial prototype and iterating quickly.

See more at [Keras.io](https://keras.io/).

## Importing the dependencies

Imagine you're starting an assignment. The only way to get information for your assignment is by going to the library and getting books which relate to your project.

This is what we're doing here. We need to use a number of things from the Keras library so we're importing (borrowing) them here.

What does the dot mean in between `keras.datasets`?

Keras is a big library. That dot means we're going into the `datasets` section of the library. Much like you would go to the science section of a regular library.

So the statement `from keras.datasets import imdb` is actually saying: Go to the datasets section of the keras library and borrow the imdb dataset.

In [ ]:
# Import the dependencies
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense, LSTM, SimpleRNN, Embedding
from keras.preprocessing import sequence
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model

print("Imported dependencies.")

Imported dependencies.


## Setting up the data

In deep learning there's often a training set and test set of data.

The training set is used for your model to learn on. Essentially, we show our model a bunch of examples and it begins to learn the patterns in those examples.

Once it knows the patterns, we can test how accurate those patterns are on the test set (a section of data the model has never seen before).

So the line,
`(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_words)` is actually saying:

Load the data from the imdb dataset and split it into a training set and a test set and make sure the maximum number of words in each set is 5000.

The imdb dataset is a dataset of 25,000 movie reviews built into the Keras library.

In [ ]:
# Define the number of words you want to use
max_words = 5000

# Define the training and test dataset
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_words)

print("Created test and training data.")

17464789/17464789 [==============================] - 0s 0us/step
Created test and training data.


## Padding the input sequences

Not every review is the same length.

Same reviews could even be one word long.

"nice"

Deep learning models look best when all of the data is in a similar shape. Imagine trying to fit 1000 different size marbles through the same size hole.

`Pad_sequences` will add 0's to any reviews which don't have a length of 500 (this is what we decided the max length to be, you can increase it).

For example, our one word review above would become:
"nice 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0... x 500"

The same goes for any reviews longer than 500 characters, they will be shortened to a maximum of 500.

So what `X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)` is saying is: Take the reviews in the X_train dataset and if they are shorter than 500 characters, add 0's to the end and if they're longer than 500 characters, cut them down to 500.

In [ ]:
# Define the maximum length of a review
max_review_length = 500

# Pad the input sequences with 0's to make them all the same length
X_train = pad_sequences(X_train, maxlen=max_review_length)
X_test = pad_sequences(X_test, maxlen=max_review_length)

print("Padded the input sequences with 0's to all be the same length.")

Padded the input sequences with 0's to all be the same length.


## Creating the model

Now our data is ready for some modelling!

Deep learning models have layers.

The top layer takes in the data we've just prepared, the middle layers do some math on this data and the final layer produces an output we can hopefully make use of.

In our case, our model has three layers, an Embedding layer, an LSTM layer and a Dense layer.

Our model begins with the line `model = Sequential()`. Think of this as simply stating "our model will flow from input to output layer in a sequential manner" or "our model goes one step at a time".

### Embedding layer

The Embedding layer makes creates a database of the relationships between words.

`model.add(Embedding(max_words, embedding_vector_length, input_length=max_review_length))` is saying: add an Embedding layer to our model and use it to turn each of our words into a list of numbers 32 digits long which have some mathematical relationship to each other.

So each of our words will become a vector, 32 digits long, of numbers.

For example, the = [0.556433, 0.223122, 0.789654....].

Don't worry for now how this is computed, Keras does it for us.

### RNN layer

`model.add(RNN(100))` is saying: add a RNN layer after our embedding layer in our model and give it 100 units.

### LSTM layer

`model.add(LSTM(100))` is saying: add a LSTM layer after our embedding layer in our model and give it 100 units.

LSTM = Long short-term memory. Think of LSTM's as a tap, a tap whichs decides which words flow through the model and which words don't. This layer uses 100 taps to decide which words matter the most in each review.

### Dense layer

`model.add(Dense(1, activation='sigmoid'))` is saying: add a Dense layer to the end of our model and use a sigmoid activation function to produce a meaningful output.

A dense layer is also known as a fully-connected layer. This layer connects the 100 LSTM units in the previous layer to 1 unit. This last unit them takes all this information and runs it through a sigmoid function.

Essentially, the sigmoid function will decide if the information should be given a 1 or a -1. 1 for positive and -1 for negative. This is will decided on based on the information passed through by the LSTM layer.


In [ ]:
# Define how long the embedding vector will be
embedding_vector_length = 32

# Define the layers in the model
model = Sequential()
model.add(Embedding(max_words, embedding_vector_length, input_length=max_review_length))
#model.add(SimpleRNN(100, return_sequences=True))
model.add(SimpleRNN(100))
#model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))

print("Model created.")

Model created.


## Compiling the model

The layers of our model our done. But we still have to put some finishing touches on it before it's ready to run.

`model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']` is saying: Stack the layers of our model on top of each other and assign a binary crossentropy loss function, use Adam for the optimizer and track accuracy metrics.

### Binary crossentropy

Think of binary crossentropy as a function which helps decide whether the output of a layer should be a 0 or 1. Binary = 0 or 1. We only want 0 or 1 as the output, because we only care about postive (1) or negative (0). If we cared about more than two categories, we would use a different loss function.

### Adam

If the model is having a hard time deciding whether an output should be 0 or 1, the Adam optimizer helps out. Adam is the name of a popular optimization function. The optimization function helps the model make better decisions on 0 or 1.

### Model metrics

Tracking the accuracy metrics will show us some live stats on how our model is doing during training (more on this soon).

In [ ]:
# Compile the model and define the loss and optimization functions
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print("Model compiled, ready to be fit to the training data.")

Model compiled, ready to be fit to the training data.


## Summarize the model

Making a summary of the model will give us an idea of what's happening at each layer.

In the embedding layer, each of our words is being turned into a list of numbers 32 digits long. Because there are 5000 words (`max_words`), there are 160,000 parameters (32 x 5000).

Parameters are individual pieces of information. The goal of the model is to take a large number of parameters and reduce them down to something we can understand and make use of (less parameters).

The LSTM layer reduces the number of parameters to 53,200 (5000 x 100 + 32 x 100).

The final dense layer connects each of the outputs of the LSTM units into one cell (100 + 1).

In [ ]:
# Summarize the different layers in the model
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 32)           160000    
                                                                 
 simple_rnn (SimpleRNN)      (None, 100)               13300     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 173401 (677.35 KB)
Trainable params: 173401 (677.35 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


## Fitting the model to the training data

Now our model is compiled, it's ready to be set loose on our training data.

We're going to run 3 cycles (`epochs=3`) on groups of 64 reviews at a time (`batch_size=64`).

Because of our loss and optimzation functions, the model accuracy should improve after each cycle.

`model.fit(X_train, y_train, epochs=3, batch_size=64)` is saying: fit the model we've built on the training dataset for 3 cycles and go over 64 reviews at a time.

Feel free to change the number of epochs (more cycles) or batch_size (more or less information each step) to see how the accuracy changes.

This will take a little time depending on how powerful your computer is. On my MacBook Pro, it took around 10-minutes.

In [ ]:
# Fit the model to the training data
model.fit(X_train, y_train, epochs=3, batch_size=64)

Epoch 1/3
391/391 [==============================] - 77s 194ms/step - loss: 0.6467 - accuracy: 0.5963
Epoch 2/3
391/391 [==============================] - 74s 190ms/step - loss: 0.4933 - accuracy: 0.7626
Epoch 3/3
391/391 [==============================] - 73s 187ms/step - loss: 0.5182 - accuracy: 0.7414


## Evaluating the model on the test data

The final step is to find out how well our trained model does on the test dataset. The data comes from the same initial library but the model has never seen it.

Think of this as studying for an exam. Your teacher tells you the exam will be on things you've learned in class. Training the model is like studying the things you've learned in class. Evaluating the model is like taking the exam.

`model_scores = model.evaluate(X_test, y_test, verbose=0)` is saying: take our trained model and see how it performs on the test dataset, we don't want the fancy progress bars so verbose is set to 0.

In [ ]:
# Evaluate the trained model on the test data
model_scores = model.evaluate(X_test, y_test, verbose=0)

# Print out the accuracy of the model on the test set
print("Model accuracy on the test dataset: {0:.2f}%".format(model_scores[1]*100))

Model accuracy on the test dataset: 81.97%


In [ ]:
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# ======= (d) epoch_4 ========


# Define how long the embedding vector will be
embedding_vector_length = 32

# Define the layers in the model
model_epoch_4 = Sequential()
model_epoch_4.add(Embedding(max_words, embedding_vector_length, input_length=max_review_length))
#model.add(SimpleRNN(100, return_sequences=True))
model_epoch_4.add(SimpleRNN(100))
#model.add(LSTM(100))
model_epoch_4.add(Dense(1, activation='sigmoid'))

print("Model_epoch_4 created.")

Model_epoch_4 created.


In [ ]:
# Compile the model and define the loss and optimization functions
model_epoch_4.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print("Model compiled, ready to be fit to the training data.")

Model compiled, ready to be fit to the training data.


In [ ]:
# Fit the model to the training data
model_epoch_4.fit(X_train, y_train, epochs=4, batch_size=64)

Epoch 1/4
391/391 [==============================] - 81s 203ms/step - loss: 0.6179 - accuracy: 0.6297
Epoch 2/4
391/391 [==============================] - 79s 202ms/step - loss: 0.5380 - accuracy: 0.7380
Epoch 3/4
391/391 [==============================] - 78s 199ms/step - loss: 0.4233 - accuracy: 0.8137
Epoch 4/4
391/391 [==============================] - 80s 206ms/step - loss: 0.3673 - accuracy: 0.8420


In [ ]:
# Evaluate the trained model on the test data
model_epoch_4_scores = model_epoch_4.evaluate(X_test, y_test, verbose=0)

model_epoch_4.save('4-(d)_1_layer_RNN100_4epochs . hdf5')

# Print out the accuracy of the model on the test set
print("Model_epoch_4 accuracy on the test dataset: {0:.2f}%".format(model_epoch_4_scores[1]*100))

Model_epoch_4 accuracy on the test dataset: 83.77%


In [ ]:
# ======= (e) epoch_5 ========

# Define how long the embedding vector will be
embedding_vector_length = 32

# Define the layers in the model
model_epoch_5 = Sequential()
model_epoch_5.add(Embedding(max_words, embedding_vector_length, input_length=max_review_length))
model_epoch_5.add(SimpleRNN(100, return_sequences=True))
model_epoch_5.add(SimpleRNN(100))
#model.add(LSTM(100))
model_epoch_5.add(Dense(1, activation='sigmoid'))

print("Model_epoch_5 created.")

Model_epoch_5 created.


In [ ]:
# Compile the model and define the loss and optimization functions
model_epoch_5.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print("Model compiled, ready to be fit to the training data.")

Model compiled, ready to be fit to the training data.


In [ ]:
# Fit the model to the training data
model_epoch_5.fit(X_train, y_train, epochs=5, batch_size=64)

Epoch 1/5
391/391 [==============================] - 181s 464ms/step - loss: 0.7015 - accuracy: 0.5326
Epoch 2/5
391/391 [==============================] - 179s 457ms/step - loss: 0.6911 - accuracy: 0.5345
Epoch 3/5
391/391 [==============================] - 180s 462ms/step - loss: 0.6660 - accuracy: 0.5776
Epoch 4/5
391/391 [==============================] - 179s 459ms/step - loss: 0.5695 - accuracy: 0.6990
Epoch 5/5
391/391 [==============================] - 178s 456ms/step - loss: 0.5414 - accuracy: 0.7234


In [ ]:
# Evaluate the trained model on the test data
model_epoch_5_scores = model_epoch_5.evaluate(X_test, y_test, verbose=0)

model_epoch_5.save('4-(d)_2_layer_RNN100_5epochs . hdf5')

# Print out the accuracy of the model on the test set
print("Model_RNN50 accuracy on the test dataset: {0:.2f}%".format(model_epoch_5_scores[1]*100))

Model_epoch_5 accuracy on the test dataset: 72.44%


In [ ]:
# ======= (e) -2 epoch_5 with one layer ========

# Define how long the embedding vector will be
embedding_vector_length = 32

# Define the layers in the model
model_epoch_5_2 = Sequential()
model_epoch_5_2.add(Embedding(max_words, embedding_vector_length, input_length=max_review_length))
model_epoch_5_2.add(SimpleRNN(100))
#model.add(LSTM(100))
model_epoch_5_2.add(Dense(1, activation='sigmoid'))

print("Model_epoch_5_2 created.")

Model_epoch_5_2 created.


In [ ]:
# Compile the model and define the loss and optimization functions
model_epoch_5_2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print("Model compiled, ready to be fit to the training data.")

In [ ]:
# Fit the model to the training data
model_epoch_5_2.fit(X_train, y_train, epochs=5, batch_size=64)

Epoch 1/5
391/391 [==============================] - 74s 189ms/step - loss: 0.3362 - accuracy: 0.8581
Epoch 2/5
391/391 [==============================] - 90s 230ms/step - loss: 0.3243 - accuracy: 0.8604
Epoch 3/5
391/391 [==============================] - 84s 216ms/step - loss: 0.4519 - accuracy: 0.7779
Epoch 4/5
391/391 [==============================] - 78s 199ms/step - loss: 0.3455 - accuracy: 0.8546
Epoch 5/5
391/391 [==============================] - 78s 200ms/step - loss: 0.2825 - accuracy: 0.8867


In [ ]:
# Evaluate the trained model on the test data
model_epoch_5_2_scores = model_epoch_5_2.evaluate(X_test, y_test, verbose=0)

# Print out the accuracy of the model on the test set
print("Model_epoch_5_2 accuracy on the test dataset: {0:.2f}%".format(model_epoch_5_2_scores[1]*100))

Model_epoch_5_2 accuracy on the test dataset: 79.86%


In [ ]:
# ======= (e) -3 epoch_3 with two layer ========

# Define how long the embedding vector will be
embedding_vector_length = 32

# Define the layers in the model
model_epoch_5_3 = Sequential()
model_epoch_5_3.add(Embedding(max_words, embedding_vector_length, input_length=max_review_length))
model_epoch_5_3.add(SimpleRNN(100, return_sequences=True))
model_epoch_5_3.add(SimpleRNN(100))
#model.add(LSTM(100))
model_epoch_5_3.add(Dense(1, activation='sigmoid'))

print("Model_epoch_5_3 created.")

Model_epoch_5_3 created.


In [ ]:
# Compile the model and define the loss and optimization functions
model_epoch_5_3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print("Model compiled, ready to be fit to the training data.")

Model compiled, ready to be fit to the training data.


In [ ]:
model_epoch_5_3.fit(X_train, y_train, epochs=3, batch_size=64)

Epoch 1/3
391/391 [==============================] - 162s 407ms/step - loss: 0.7067 - accuracy: 0.5054
Epoch 2/3
391/391 [==============================] - 160s 409ms/step - loss: 0.6458 - accuracy: 0.6119
Epoch 3/3
391/391 [==============================] - 159s 406ms/step - loss: 0.5586 - accuracy: 0.7050


In [ ]:
# Evaluate the trained model on the test data
model_epoch_5_3_scores = model_epoch_5_3.evaluate(X_test, y_test, verbose=0)

# Print out the accuracy of the model on the test set
print("Model_epoch_5_2 accuracy on the test dataset: {0:.2f}%".format(model_epoch_5_3_scores[1]*100))

Model_epoch_5_2 accuracy on the test dataset: 63.94%


In [ ]:
# Define how long the embedding vector will be
embedding_vector_length = 32

# Define the layers in the model
model_RNN50 = Sequential()
model_RNN50.add(Embedding(max_words, embedding_vector_length, input_length=max_review_length))
model_RNN50.add(SimpleRNN(50))
#model.add(LSTM(100))
model_RNN50.add(Dense(1, activation='sigmoid'))

print("Model_RNN50 created.")

Model_RNN50 created.


In [ ]:
# Compile the model and define the loss and optimization functions
model_RNN50.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print("Model compiled, ready to be fit to the training data.")

Model compiled, ready to be fit to the training data.


In [ ]:
# Fit the model to the training data
model_RNN50.fit(X_train, y_train, epochs=3, batch_size=64)

Epoch 1/3
391/391 [==============================] - 59s 147ms/step - loss: 0.6024 - accuracy: 0.6503
Epoch 2/3
391/391 [==============================] - 55s 140ms/step - loss: 0.5947 - accuracy: 0.6917
Epoch 3/3
391/391 [==============================] - 55s 141ms/step - loss: 0.5283 - accuracy: 0.7472


In [ ]:
# Evaluate the trained model on the test data
model_RNN50_scores = model_RNN50.evaluate(X_test, y_test, verbose=0)

model_RNN50.save('4-(f)_1_layer_RNN50_3epochs . hdf5')

# Print out the accuracy of the model on the test set
print("Model_RNN50 accuracy on the test dataset: {0:.2f}%".format(model_RNN50_scores[1]*100))

Model_RNN50 accuracy on the test dataset: 75.06%


In [ ]:
# Define how long the embedding vector will be
embedding_vector_length = 32

# Define the layers in the model
model_RNN50_2 = Sequential()
model_RNN50_2.add(Embedding(max_words, embedding_vector_length, input_length=max_review_length))
model_RNN50_2.add(SimpleRNN(50, return_sequences=True))
model_RNN50_2.add(SimpleRNN(50))
#model.add(LSTM(100))
model_RNN50_2.add(Dense(1, activation='sigmoid'))

print("model_RNN50_2 created.")

model_RNN50_2 created.


In [ ]:
# Compile the model and define the loss and optimization functions
model_RNN50_2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print("Model compiled, ready to be fit to the training data.")

Model compiled, ready to be fit to the training data.


In [ ]:
# Fit the model to the training data
model_RNN50_2.fit(X_train, y_train, epochs=3, batch_size=64)

Epoch 1/3
391/391 [==============================] - 146s 335ms/step - loss: 0.5560 - accuracy: 0.6938
Epoch 2/3
391/391 [==============================] - 128s 327ms/step - loss: 0.4448 - accuracy: 0.7891
Epoch 3/3
391/391 [==============================] - 127s 324ms/step - loss: 0.4900 - accuracy: 0.7633


In [ ]:
# Evaluate the trained model on the test data
model_RNN50_2_scores = model_RNN50_2.evaluate(X_test, y_test, verbose=0)

model_RNN50_2.save('4-(g)_2_layer_RNN50_3epochs . hdf5')

# Print out the accuracy of the model on the test set
print("Model_RNN50_2 accuracy on the test dataset: {0:.2f}%".format(model_RNN50_2_scores[1]*100))

In [ ]:
# Define how long the embedding vector will be
embedding_vector_length = 32

# Define the layers in the model
model_LSTM = Sequential()
model_LSTM.add(Embedding(max_words, embedding_vector_length, input_length=max_review_length))
model_LSTM.add(LSTM(100))
model_LSTM.add(Dense(1, activation='sigmoid'))

print("model_LSTM created.")

model_LSTM created.


In [ ]:
# Compile the model and define the loss and optimization functions
model_LSTM.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print("Model compiled, ready to be fit to the training data.")

Model compiled, ready to be fit to the training data.


In [ ]:
# Fit the model to the training data
model_LSTM.fit(X_train, y_train, epochs=3, batch_size=64)

Epoch 1/3
391/391 [==============================] - 275s 696ms/step - loss: 0.4927 - accuracy: 0.7500
Epoch 2/3
391/391 [==============================] - 261s 669ms/step - loss: 0.3498 - accuracy: 0.8496
Epoch 3/3
391/391 [==============================] - 262s 671ms/step - loss: 0.2690 - accuracy: 0.8940


In [ ]:
# Evaluate the trained model on the test data
model_LSTM_scores = model_LSTM.evaluate(X_test, y_test, verbose=0)

model_LSTM.save('4-(h)_1_layer_LSTM_3epochs . hdf5')

# Print out the accuracy of the model on the test set
print("Model_LSTM accuracy on the test dataset: {0:.2f}%".format(model_LSTM_scores[1]*100))

Model_LSTM accuracy on the test dataset: 86.57%


## Saving the model


In [ ]:
!mkdir -p checkpoints

# save your model
model.save('./checkpoints/my_checkpoint')
print('saved')

# validate your saved model
new_model = load_model('./checkpoints/my_checkpoint')
print('loaded')

saved
loaded
